In [1]:
import pandas as pd
import numpy as np
import datetime
  
from numpy import isnan

import warnings
warnings.filterwarnings("ignore")

In [2]:
#legally operating business
#download the data
!curl 'https://data.cityofnewyork.us/api/views/w7w3-xahh/rows.csv?accessType=DOWNLOAD' > legally_operating_business.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 59.9M    0 59.9M    0     0  3956k      0 --:--:--  0:00:15 --:--:-- 4212k  0  3813k      0 --:--:--  0:00:08 --:--:-- 3962k   0 --:--:--  0:00:14 --:--:-- 4118k


In [3]:
business_df = pd.read_csv ('legally_operating_business.csv', low_memory=False)

In [4]:
#convert datetime to datetime object
business_df['License Creation Date'] = pd.to_datetime(business_df['License Creation Date'])
business_df['License Expiration Date'] = pd.to_datetime(business_df['License Expiration Date'])

In [5]:
#extract start_date
business_df['year'] = business_df['License Creation Date'].dt.year
business_df['month'] = business_df['License Creation Date'].dt.month
business_df['day'] = 1
business_df['start_date'] = pd.to_datetime(business_df[['year', 'month','day']])

In [6]:
#extract end_date
business_df['year'] = business_df['License Expiration Date'].dt.year
business_df['month'] = business_df['License Expiration Date'].dt.month
business_df['day'] = 1
business_df['end_date'] = pd.to_datetime(business_df[['year', 'month','day']])

In [7]:
#dataframe for business that is active on 2016-11-01
business_df[(business_df['start_date'].dt.year == 2016) & (business_df['start_date'].dt.month == 11)]

,DCA License Number,License Type,License Expiration Date,License Status,License Creation Date,Industry,Business Name,Business Name 2,Address Building,Address Street Name,...,Census Tract,Detail,Longitude,Latitude,Location,year,month,day,start_date,end_date
135,2045761-DCA,Business,2019-03-15,Inactive,2016-11-21,Dealer In Products,International Medical Equipment LLC,HELP MEDICAL SUPPLIES,57,W 57TH ST,...,11201.0,NaN,-73.975741,40.763719,"(40.76371877676414, -73.97574145656893)",2019.0,3.0,1,2016-11-01,2019-03-01
144,2045842-DCA,Business,2023-02-28,Active,2016-11-22,Home Improvement Contractor,VMW CONSTRUCTION DELUXE LLC,NaN,29,WASHINGTON ST,...,NaN,NaN,NaN,NaN,NaN,2023.0,2.0,1,2016-11-01,2023-02-01
190,2045884-DCA,Business,2019-12-26,Inactive,2016-11-25,Sidewalk Cafe,MOTORINO 3 INC,NaN,510,COLUMBUS AVE,...,169.0,"Sidewalk Cafe Type: Unenclosed, Square Feet: 1...",-73.972895,40.785517,"(40.78551685027273, -73.97289520236131)",2019.0,12.0,1,2016-11-01,2019-12-01
785,2045226-DCA,Business,2023-02-28,Active,2016-11-02,Home Improvement Contractor,"JDP DESIGN CONSTRUCTION, INC.",NaN,142,HENRY ST,...,NaN,NaN,NaN,NaN,NaN,2023.0,2.0,1,2016-11-01,2023-02-01
847,2045785-DCA,Business,2020-12-31,Inactive,2016-11-21,Electronics Store,"AT&T MOBILITY SUPPLY, LLC",AT&T Mobility,30,ROCKEFELLER PLZ,...,104.0,NaN,-73.978692,40.758754,"(40.75875411971916, -73.97869233509232)",2020.0,12.0,1,2016-11-01,2020-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272962,2045254-DCA,Individual,NaT,Inactive,2016-11-02,Home Improvement Salesperson,"TRAINER, CRAIG",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2016-11-01,NaT
273166,2045467-DCA,Individual,2019-07-31,Inactive,2016-11-10,Ticket Seller,"HENK, GABRIELA E.",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2019.0,7.0,1,2016-11-01,2019-07-01
273191,2045724-DCA,Individual,2019-02-28,Inactive,2016-11-18,Home Improvement Salesperson,"KISSOON, PAUL",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2019.0,2.0,1,2016-11-01,2019-02-01
273227,2045222-DCA,Individual,2019-07-31,Inactive,2016-11-02,Ticket Seller,"TONYIGBE, KOSSI",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2019.0,7.0,1,2016-11-01,2019-07-01


In [8]:
#community district code
cd = [101., 102., 103., 104., 105., 106., 107., 108., 109., 110., 111.,
       112., 201., 202., 203., 204., 205., 206., 207., 208., 209., 210.,
       211., 212., 301., 302., 303., 304., 305., 306., 307., 308.,
       309., 310., 311., 312., 313., 314., 315., 316., 317., 318., 401.,
       402., 403., 404., 405., 406., 407., 408., 409., 410., 411., 412.,
       413., 414., 501., 502., 503.]

column_names = ["borocd", "business", "month"]

business_agg = pd.DataFrame(columns = column_names)

start_month = '2016-11-01'
end_month = '2021-10-01'
month_df = pd.DataFrame(pd.date_range(start_month,end_month, 
              freq='MS').strftime("%Y-%m-01").tolist(),columns=['month'])
month_df['month'] = pd.to_datetime(month_df['month'])

In [9]:
def calc_business(date_month):
  global business_agg
  date = date_month #datetime.datetime(2017, 1, 1)
  x = business_df[(business_df['start_date'] < date) & (business_df['end_date'] > date)].groupby('Community Board').agg({'Business Name':'count'}).reset_index().rename(columns={"Community Board": "borocd", "Business Name": "business"}, errors="raise")
  x['month'] = date
  x = x[x.borocd.isin(cd)].reset_index(drop=True)
  business_agg = pd.concat([business_agg, x])
    #business_agg.append(x)

In [10]:
for month in month_df['month']:
  calc_business(month)

In [11]:
business_agg

,borocd,business,month
0,101.0,683,2016-11-01 00:00:00
1,102.0,1013,2016-11-01 00:00:00
2,103.0,947,2016-11-01 00:00:00
3,104.0,1014,2016-11-01 00:00:00
4,105.0,2083,2016-11-01 00:00:00
...,...,...,...
54,413.0,637,2021-10-01 00:00:00
55,414.0,235,2021-10-01 00:00:00
56,501.0,828,2021-10-01 00:00:00
57,502.0,691,2021-10-01 00:00:00


## **PLUTO**

Create a template for pluto aggregrate

In [12]:
column_names = ["cd", "numbldgs", "unitstotal", "asland_sqft", "pluto_ver"]
pluto_agg = pd.DataFrame(columns = column_names)

In [13]:
pluto_df = pd.read_csv ('pluto_21v4.csv', low_memory=False).groupby('cd').agg({'assessland':'sum','lotarea':'sum','numbldgs':'sum','unitstotal':'sum'}).reset_index()
pluto_df['asland_sqft'] = pluto_df['assessland'] / pluto_df['lotarea']
pluto_df = pluto_df[pluto_df.cd.isin(cd)].reset_index(drop=True)
pluto_df['pluto_ver'] = '21v4'
pluto_agg = pluto_agg.append(pluto_df[['cd','numbldgs','unitstotal','asland_sqft','pluto_ver']])

In [14]:
pluto_df = pd.read_csv ('pluto_21v3.csv', low_memory=False).groupby('cd').agg({'assessland':'sum','lotarea':'sum','numbldgs':'sum','unitstotal':'sum'}).reset_index()
pluto_df['asland_sqft'] = pluto_df['assessland'] / pluto_df['lotarea']
pluto_df = pluto_df[pluto_df.cd.isin(cd)].reset_index(drop=True)
pluto_df['pluto_ver'] = '21v3'
pluto_agg = pluto_agg.append(pluto_df[['cd','numbldgs','unitstotal','asland_sqft','pluto_ver']])

In [15]:
def pluto_cleaning(version):
  global pluto_agg
  pluto_df = pd.read_csv ('pluto_' + version + '.csv', low_memory=False).groupby('cd').agg({'assessland':'sum','lotarea':'sum','numbldgs':'sum','unitstotal':'sum'}).reset_index()
  pluto_df['asland_sqft'] = pluto_df['assessland'] / pluto_df['lotarea']
  pluto_df = pluto_df[pluto_df.cd.isin(cd)].reset_index(drop=True)
  pluto_df['pluto_ver'] = version
  pluto_agg = pluto_agg.append(pluto_df[['cd','numbldgs','unitstotal','asland_sqft','pluto_ver']])

In [16]:
pluto_versions = ['18v11','18v2_1','19v1','19v2','20v1','20v2','20v3','20v4','20v6','20v7','20v8','21v1','21v2','21v3','21v4']

for version in pluto_versions:
  pluto_cleaning(version)

In [163]:
pluto_agg = pluto_agg.reset_index(drop=True)

In [18]:
pluto_agg.to_csv('pluto.csv')

Community District Built Environment

In [19]:
!curl 'https://raw.githubusercontent.com/NYCPlanning/db-community-profiles/main/cdprofiles_build/output/cd_built_environment.csv' > cd_built_environment.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14006  100 14006    0     0  59837      0 --:--:-- --:--:-- --:--:-- 59854


In [20]:
built_df = pd.read_csv ('cd_built_environment.csv', low_memory=False)

In [21]:
built_df = built_df[['cd_number','pct_clean_strts','pct_lot_area_open_space','pct_lot_area_public_facility_institution','pct_lot_area_res_1_2_family_bldg','pct_lot_area_res_multifamily_walkup']]

In [22]:
built_df = built_df.rename(columns={"cd_number": "borocd"}).sort_values(by=['borocd'])

In [23]:
built_df = built_df.sort_values(by=['borocd']).reset_index(drop=True)

In [108]:
column_names = ["borocd", "month"]
built_agg = pd.DataFrame(columns = column_names)

for month in month_df['month']:
    built_df['month'] = month.strftime("%Y-%m-%d")
    built_agg = pd.concat([built_agg, built_df])

built_agg['month'] = pd.to_datetime(built_agg['month'])

In [109]:
built_agg

,borocd,month,pct_clean_strts,pct_lot_area_open_space,pct_lot_area_public_facility_institution,pct_lot_area_res_1_2_family_bldg,pct_lot_area_res_multifamily_walkup
0,101,2016-11-01,100.0,4.35,31.68,0.17,0.63
1,102,2016-11-01,98.7,3.55,9.63,4.47,10.26
2,103,2016-11-01,93.4,14.95,11.15,0.32,8.66
3,104,2016-11-01,100.0,2.81,11.75,0.88,7.68
4,105,2016-11-01,100.0,1.59,6.49,0.09,0.67
...,...,...,...,...,...,...,...
54,413,2021-10-01,98.1,11.31,7.98,61.24,5.19
55,414,2021-10-01,93.8,18.39,5.12,41.66,4.15
56,501,2021-10-01,99.1,9.62,7.36,43.86,4.77
57,502,2021-10-01,100.0,21.05,10.92,27.85,2.17


In [26]:
built_df.to_csv('built_env.csv')

In [27]:
business_agg.to_csv('business_agg.csv')

In [ ]:
#sidewalk_df = pd.read_csv ('sidewalk.csv', low_memory=False)
#sidewalk_df[~sidewalk_df['ISSUANCE_DD'].isna()]['EXPIRATION_DATE']
#sidewalk_issued = sidewalk_df[sidewalk_df['ISSUANCE'] == 'Issued']
#sidewalk_issued['EXPIRATION_DATE'] = pd.to_datetime(sidewalk_issued['EXPIRATION_DATE'])
#sidewalk_issued['ISSUANCE_DD'] = pd.to_datetime(sidewalk_issued['ISSUANCE_DD'])

In [ ]:
# sidewalk_issued['year'] = sidewalk_issued['ISSUANCE_DD'].dt.year
# sidewalk_issued['month'] = sidewalk_issued['ISSUANCE_DD'].dt.month
# sidewalk_issued['day'] = 1
# sidewalk_issued['start_date'] = pd.to_datetime(sidewalk_issued[['year', 'month','day']])

In [ ]:
# sidewalk_issued['year'] = sidewalk_issued['EXPIRATION_DATE'].dt.year
# sidewalk_issued['month'] = sidewalk_issued['EXPIRATION_DATE'].dt.month
# sidewalk_issued['day'] = 1
# sidewalk_issued['end_date'] = pd.to_datetime(sidewalk_issued[['year','month','day']])

In [ ]:
# sidewalk_issued['']

In [ ]:
# sidewalk_agg = pd.DataFrame(columns = column_names)

In [ ]:
# def calc_sidewalk(date_month):
#   global sidewalk_agg
#   date = date_month #datetime.datetime(2017, 1, 1)
#   x = sidewalk_issued[(sidewalk_issued['start_date'] < date) & (sidewalk_issued['end_date'] > date)].groupby('COMMUNITY_DISTRICT').agg({'BUSINESS_NAME':'count'}).reset_index().rename(columns={"COMMUNITY_DISTRICT": "borocd", "BUSINESS_NAME": "business"}, errors="raise")
#   x['month'] = date
#   x = x[x.borocd.isin(cd)].reset_index(drop=True)
#   sidewalk_agg = sidewalk_agg.append(x)

In [ ]:
# for month in month_df['month']:
#   calc_sidewalk(month)

In [ ]:
# sidewalk_agg

In [29]:
open_restaurant = pd.read_csv ('https://data.cityofnewyork.us/api/views/4dx7-axux/rows.csv?accessType=DOWNLOAD', low_memory=False) 

In [72]:
restaurant = open_restaurant[['Borough','RestaurantName','CommunityBoard']]

In [73]:
#finding restaurant with community board value
restaurant = restaurant[~isnan(restaurant.CommunityBoard)]

In [76]:
boro_dict = {'Manhattan':'MN','Queens':'QN','Brooklyn':'BK','Bronx':'BX','Staten Island':'SI'}
restaurant['BoroughID'] = restaurant['Borough'].replace(boro_dict)

In [79]:
restaurant['cb'] = ["%02d" % x for x in restaurant.CommunityBoard]
restaurant['community_district'] = restaurant.BoroughID + restaurant.cb

In [89]:
restaurant = restaurant.groupby('community_district').agg({'RestaurantName':'count'}).reset_index().rename(columns={'RestaurantName':'restaurant'})

In [113]:
restaurant = restaurant.drop([56]).reset_index(drop=True)

In [114]:
column_names = ["community_district", "restaurant", "month"]

restaurant_agg = pd.DataFrame(columns = column_names)

In [115]:
#put month data
for month in month_df['month']:
    restaurant['month'] = month.strftime("%Y-%m-%d")
    restaurant_agg = pd.concat([restaurant_agg, restaurant])

In [116]:
restaurant_agg

,community_district,restaurant,month
0,BK01,3295,2016-11-01
1,BK02,1271,2016-11-01
2,BK03,580,2016-11-01
3,BK04,964,2016-11-01
4,BK05,270,2016-11-01
...,...,...,...
54,QN13,63,2021-10-01
55,QN14,120,2021-10-01
56,SI01,161,2021-10-01
57,SI02,100,2021-10-01


In [144]:
built_agg['cd']=built_agg['borocd'].apply(str).str[1:]

In [146]:
built_agg['boro']=built_agg['borocd'].apply(str).str[0]

In [151]:
boro_dict2 = {'1':'MN','2':'QN','4':'BK','3':'BX','5':'SI'}
built_agg['boro'] = built_agg['boro'].replace(boro_dict2)

In [153]:
built_agg['community_district'] = built_agg['boro'] + built_agg['cd']

In [155]:
built_agg

,borocd,month,pct_clean_strts,pct_lot_area_open_space,pct_lot_area_public_facility_institution,pct_lot_area_res_1_2_family_bldg,pct_lot_area_res_multifamily_walkup,cd,boro,community_district
0,101,2016-11-01,100.0,4.35,31.68,0.17,0.63,01,MN,MN01
1,102,2016-11-01,98.7,3.55,9.63,4.47,10.26,02,MN,MN02
2,103,2016-11-01,93.4,14.95,11.15,0.32,8.66,03,MN,MN03
3,104,2016-11-01,100.0,2.81,11.75,0.88,7.68,04,MN,MN04
4,105,2016-11-01,100.0,1.59,6.49,0.09,0.67,05,MN,MN05
...,...,...,...,...,...,...,...,...,...,...
54,413,2021-10-01,98.1,11.31,7.98,61.24,5.19,13,BK,BK13
55,414,2021-10-01,93.8,18.39,5.12,41.66,4.15,14,BK,BK14
56,501,2021-10-01,99.1,9.62,7.36,43.86,4.77,01,SI,SI01
57,502,2021-10-01,100.0,21.05,10.92,27.85,2.17,02,SI,SI02


In [174]:
pluto_agg['cd2']=pluto_agg['cd'].apply(str).str[1:3]
pluto_agg['boro']=pluto_agg['cd'].apply(str).str[0]
pluto_agg['boro'] = pluto_agg['boro'].replace(boro_dict2)
pluto_agg['community_district'] = pluto_agg['boro'] + pluto_agg['cd2']

In [175]:
pluto_agg

,cd,numbldgs,unitstotal,asland_sqft,pluto_ver,cd2,boro,community_district
0,101.0,1698.0,54929.0,136.726968,21v4,01,MN,MN01
1,102.0,5234.0,72087.0,121.199995,21v4,02,MN,MN02
2,103.0,4631.0,93366.0,67.106492,21v4,03,MN,MN03
3,104.0,3820.0,101391.0,109.335751,21v4,04,MN,MN04
4,105.0,3075.0,91782.0,515.656879,21v4,05,MN,MN05
...,...,...,...,...,...,...,...,...
998,413.0,70667.0,65968.0,5.963269,21v4,13,BK,BK13
999,414.0,20858.0,48914.0,5.004623,21v4,14,BK,BK14
1000,501.0,49314.0,75769.0,4.359251,21v4,01,SI,SI01
1001,502.0,40211.0,54370.0,4.431627,21v4,02,SI,SI02


In [176]:
pluto_agg.to_csv('pluto.csv')
built_agg.to_csv('built.csv')
restaurant_agg.to_csv('restaurant.csv')

In [177]:
business_agg.to_csv('business.agg')